## Wrangle Micro Data Notebook

In [1]:
import time
import requests
import pandas as pd
from datetime import datetime

import wrangle_rapid as w
import wrangle_hud_micro as whm
from env import rm_api, hud_token
import warnings
warnings.filterwarnings('ignore')


### Rapid API Micro Data

In [2]:
corey_zipcodes = ['78258', '78259', '78260', '78261', '78263', '78264', '78265',
       '78266', '78268', '78269', '78270', '78278', '78279', '78280',
       '78283', '78285', '78288', '78291', '78292', '78293', '78294',
       '78295', '78296', '78297', '78298', '78299', '78606', '78623',
       '78638', '78648', '78655', '78666', '78670', '78676', '78883',
       '78884', '78885', '78117', '78850']

In [3]:
# w.get_rapid_api(corey_zipcodes, rm_api, 'baughman_rapid_api.csv')

In [4]:
df_rapid = w.rapid_api_mvp_prep('baughman_rapid_api.csv')

In [5]:
df_rapid.head(1)

,bedrooms,averageRent,minRent,maxRent,totalRentals,zipcode
date,,,,,,
2020-04-01,2,1648.78,1303,2495,9,78261


In [6]:
cols = df_rapid.columns.to_list()

In [7]:
new_cols = ['bedrooms', 'average_rent', 'min_rent',
            'max_rent', 'num_properties', 'zip_code']

In [8]:
df_rapid = df_rapid.rename(columns = dict(zip(cols, new_cols)))

### HUD Micro Data

In [9]:
# gonna do some quick validation on the original HUD dataset pulled

In [10]:
df_1 = whm.get_yearly_entity_data('METRO41700M41700')
df_1.head(1)

,zip_code,Efficiency,One-Bedroom,Two-Bedroom,Three-Bedroom,Four-Bedroom,year,entity_id,area_name
0,78002,570,710,880,1170,1420,2018,METRO41700M41700,"San Antonio-New Braunfels, TX HUD Metro FMR Area"


In [24]:
# okay, now checking number of zip codes
df_1.zip_code.value_counts()

78002    6
78245    6
78259    6
78258    6
78257    6
        ..
78207    6
78241    3
78285    3
78850    1
78117    1
Name: zip_code, Length: 138, dtype: int64

In [ ]:
# looks like there are 138 zip codes present
# there are 6 years of data
# four of those have incomplete entries. Why?
# 

In [29]:
df_1[df_1.zip_code == '78117']

,zip_code,Efficiency,One-Bedroom,Two-Bedroom,Three-Bedroom,Four-Bedroom,year,entity_id,area_name
23,78117,660,800,1000,1310,1610,2019,METRO41700M41700,"San Antonio-New Braunfels, TX HUD Metro FMR Area"


Missing ZIP codes:
- 78241 only belongs to the MSA in 2021, 2022, 2023
- 78285 only belongs to the MSA in 2021, 2022, 2023
- 78850 only belongs to the MSA in 2018
- 78117 only belongs to the MSA in 2019


In [11]:
df_hud = whm.get_hud_micro_data()

In [12]:
whm.get_hud_micro_data()

,zip_code,two_bed_fmr
date,,
2017-10-01,78002,880.0
2017-11-01,78002,880.0
2017-12-01,78002,880.0
2018-01-01,78002,880.0
2018-02-01,78002,880.0
...,...,...
2023-06-01,78285,1270.0
2023-07-01,78285,1270.0
2023-08-01,78285,1270.0


In [13]:
# df_hud.index = pd.to_datetime(df_hud.index)

In [14]:
df_hud

,zip_code,two_bed_fmr
date,,
2017-10-01,78002,880.0
2017-11-01,78002,880.0
2017-12-01,78002,880.0
2018-01-01,78002,880.0
2018-02-01,78002,880.0
...,...,...
2023-06-01,78285,1270.0
2023-07-01,78285,1270.0
2023-08-01,78285,1270.0


In [15]:
df_hud.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9990 entries, 2017-10-01 to 2023-10-01
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   zip_code     9990 non-null   int64  
 1   two_bed_fmr  9852 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 234.1+ KB


In [16]:
df_hud.index.value_counts()

2020-10-01    138
2020-11-01    138
2022-03-01    138
2022-02-01    138
2022-01-01    138
             ... 
2018-06-01    135
2018-07-01    135
2018-08-01    135
2018-09-01    135
2017-10-01    135
Name: date, Length: 73, dtype: int64

In [17]:
df_hud[df_hud.zip_code == '78285']

,zip_code,two_bed_fmr
date,,


In [18]:
df_hud.head()

,zip_code,two_bed_fmr
date,,
2017-10-01,78002,880.0
2017-11-01,78002,880.0
2017-12-01,78002,880.0
2018-01-01,78002,880.0
2018-02-01,78002,880.0


In [19]:
df_hud.tail()

,zip_code,two_bed_fmr
date,,
2023-06-01,78285,1270.0
2023-07-01,78285,1270.0
2023-08-01,78285,1270.0
2023-09-01,78285,1270.0
2023-10-01,78285,NaN


In [20]:
# # ok, I need to clean this up. keep just the Two-Bedroom col as two_bed_fmr
# # and year, entity_id, and area_name
# df_hud = df_hud.rename(columns={'Two-Bedroom' : 'two_bed_fmr'})
# df_hud = df_hud[['zip_code', 'two_bed_fmr', 'year', 'entity_id', 'area_name']]
# df_hud['date'] = pd.to_datetime(df_hud.year)
# df_hud.set_index('date', inplace=True)
# # shift the fmr rates backwards by three months to reflect federal fiscal year
# df_hud = df_hud.resample('M').ffill().shift(periods=-3, freq='M')

In [22]:
# Perform an inner join on 'date' and 'zip_code'
# df = df_rapid.merge(df_hud, how='inner', on=['date', 'zip_code'])